# Share of Trip Mode by State
In this notebook, we have three main tasks: <br />
1) Use the 2017 National Household Travel Survey to plot out the share of trips with different mode in each state <br/>
2) Use API to download census data and map out some demographic and income information by state<br/>
3) Create a webapp using Panel that allows user to select mode choice and demographic information with maps and a bar chart that can update themselves automatically.

In [ ]:
# Let's setup the imports we'll need first
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import geopandas as gpd

%matplotlib inline

import hvplot.pandas
import altair as alt

import holoviews as hv
from holoviews import opts
hv.extension("bokeh")

from datashader.utils import lnglat_to_meters

In [ ]:
from datashader.utils import lnglat_to_meters
# Get USA xlim and ylim in meters (EPSG=3857)
USA = ((-124.72,  -66.95), (23.55, 50.06))
USA_xlim_meters, USA_ylim_meters = [list(r) for r in lnglat_to_meters(USA[0], USA[1])]

In [ ]:
# Read in NHTS data
# trip = pd.read_csv("./NHTS/Data/trippub.csv")
# person = pd.read_csv("./NHTS/Data/perpub.csv")

## Trip Mode Share Map
Firstly, use the trip dataset, group by household state and by mode choice and add up the trip numbers taking weight into consideration.
<br />
Secondly, group by state, calculate the count of trips for each mode choice and total trips, and then calculate the share of trips with each mode choice. 
<br />
Thirdly, add geometry feature to the trip information and plot with Hvplot groupby trip mode to obtain a dropdown bar.

In [ ]:
# group by state and mode choice
# trip_state_mode = trip.groupby(['HHSTATE','TRPTRANS','HHSTFIPS'])['WTTRDFIN'].sum().reset_index()
# trip_state_mode

Our goal is to have a column for percentage of trips by each travel mode for each state. So we need to make some changes and calculations to the current dataset. We first calculate total number of trips in each state and then join the total number to the above dataframe which was grouped by state and by modes of transportation. So each row of observation will have the count of total trips in that state and the count of trips of a specific mode in that state.

In [ ]:
# # create a new df group by state to calculate total trips in a state
# trip_state = trip.groupby(['HHSTATE','HHSTFIPS'])['WTTRDFIN'].sum().reset_index()
# trip_state = trip_state.rename(columns={'WTTRDFIN': 'state_w'}) # rename columns to make more intuitive
# trip_state_mode = trip_state_mode.rename(columns={'WTTRDFIN': 'state_mode_w'})

# # merge df groupped by state&mode with df groupped by state
# trip_state_mode = trip_state_mode.merge(trip_state[['HHSTATE','state_w']], left_on='HHSTATE', right_on='HHSTATE')
# trip_state_mode['mode_share'] = trip_state_mode['state_mode_w']/trip_state_mode['state_w']
# trip_state_mode

We will not be considering mode choice -9,-8,-7 which are Not ascertained, I don't know and I prefer not to answer when making the final hvplot. So we will filter them out before moving into plotting steps.

In [ ]:
# a = [-9,-8,-7] 
# trip_state_mode= trip_state_mode[~trip_state_mode['TRPTRANS'].isin(a)] # filter out no answers
# trip_state_mode

Note that since not all travel mode choice appeared in each state. The above trip_state_mode dataframe does not contain all possible combination of state&mode pairs. Since if a state does not have a specific mode, the geometry will not appear in the map, we need to add these rows into the dataframe and specify the share and weight as 0.

In [ ]:
# # create empty dataframe for each state and trip mode and indicator of 0.
# state_list = trip_state_mode['HHSTATE'].unique().tolist()
# mode_list = trip_state_mode['TRPTRANS'].unique().tolist()

# rows = []
# for i in range(len(state_list)):
#     for j in range(len(mode_list)):
#         rows.append([state_list[i], mode_list[j], 0])
# empty = pd.DataFrame(rows, columns=["state", "mode", "indicator"])
# empty

In [ ]:
# # merge empty df with trip_state_mode_geo to get a dataframe with all possible combination of state and mode choice
# trip_state_mode_all = trip_state_mode.merge(empty, left_on=['HHSTATE','TRPTRANS'], right_on=['state','mode'], 
#                                             how='right')
# trip_state_mode_all

Note that after this step, we have all possible state&mode pairs. The pairs that did not appear in the original trip dataframe will have NAs in the associated share column. So we need to fill in these values as 0.

In [ ]:
# # Fill in the NAs from the above dataframe
# trip_state_mode_all['TRPTRANS'] = trip_state_mode_all['TRPTRANS'].fillna(trip_state_mode_geoall['mode'])
# trip_state_mode_all['HHSTATE'] = trip_state_mode_all['HHSTATE'].fillna(trip_state_mode_geoall['state'])
# trip_state_mode_all['mode_share'] = trip_state_mode_all['mode_share'].fillna(0)

# trip_state_mode_all

Now we have our trip data ready. We can get the geographical information and merge the geometries of each state with our curret trip_state_mode df. Note that here we explored one way of adding the state boundaries -- getting geojson file online. In the next section, we also tried getting geojson using cenpy API.

In [ ]:
# states= gpd.read_file("./NHTS/Data/gz_2010_us_040_00_500k.json")

In [ ]:
# # first plot out the geographical informaiton of US

# fig, ax = plt.subplots(figsize=(60, 6))
# ax = states.plot(ax=ax, facecolor='none', edgecolor='black')
# ax.set_axis_off()

In [ ]:
# states.head()

In [ ]:
# # convert state code from object to integer to prepare for joining with Household Travel Survey data
# states['STATE'] = states['STATE'].astype('int32')

In [ ]:
# # merge states with trip_state_mode by state fips code
# states = states.merge(trip_state_mode[['HHSTFIPS','HHSTATE']].drop_duplicates(),
#                       left_on='STATE', right_on='HHSTFIPS', how='left')
# states.head()

In [ ]:
# trip_state_mode_geo = trip_state_mode_all.merge(states, left_on='state', right_on='HHSTATE', how='left')
# trip_state_mode_geo.head()

Now we have both datasets ready. We can use hvplot to make some interactive plots.

In [ ]:
# trip_state_mode_geo = gpd.GeoDataFrame(trip_state_mode_geo, geometry='geometry', crs="EPSG:4326")

In [ ]:
# # change some column names and value names to make plot more interpretatble
# def label_mode (row):
#     a = row['TRPTRANS']
#     if a == 1 :return 'Walk'
#     if a == 2 :return 'Bicycle'
#     if a == 3 :return 'Car'
#     if a == 4 :return 'SUV'
#     if a == 5 :return 'Van'
#     if a == 6 :return 'Pickup truck'
#     if a == 7 :return 'Golf cart/Segway'
#     if a == 8 :return 'Motorcycle/Moped'
#     if a == 9 :return 'RV (motor home, ATV, snowmobile)'
#     if a == 10 :return 'School bus'
#     if a == 11 :return 'Public or commuter bus'
#     if a == 12 :return 'Paratransit/Dial-a-ride'
#     if a == 13 :return 'Private/Charter/Tour/Shuttle bus'
#     if a == 14 :return 'City-to-city bus (Greyhound, Megabus)'
#     if a == 15 :return 'Amtrak/Commuter rail'
#     if a == 16 :return 'Subway/elevated/light rail/street car'
#     if a == 17 :return 'Taxi/limo (Uber/Lyft)'
#     if a == 18 :return 'Rental car'
#     if a == 19 :return 'Airplane'
#     if a == 20 :return 'Boat/ferry/water taxi'
#     if a == 97 :return 'Something Else'

# trip_state_mode_geo['Trip Mode'] = trip_state_mode_geo.apply (lambda row: label_mode(row), axis=1)
# trip_state_mode_geo.head()

In [ ]:
# # We experienced some difficulties setting boundaries for the hvplot below using crs=4326, 
# # so we converted it to 3857 here. Not sure what's the problem with 4326.

# trip_state_mode_geo = trip_state_mode_geo.to_crs(epsg=3857)
# trip_state_mode_geo.crs

In [ ]:
# from datashader.utils import lnglat_to_meters
# # Get USA xlim and ylim in meters (EPSG=3857)
# USA = ((-124.72,  -66.95), (23.55, 50.06))
# USA_xlim_meters, USA_ylim_meters = [list(r) for r in lnglat_to_meters(USA[0], USA[1])]

# mode_map = trip_state_mode_geo.hvplot.polygons(c='mode_share', 
#                                                frame_width=400, 
#                                                frame_height=300, 
#                                                groupby="Trip Mode",
#                                                geo=True, 
#                                                cmap='RdPu', 
#                                                crs=3857,
#                                                hover_cols=['state'],
#                                                xlim = USA_xlim_meters, #add boundaries to make map centered on the U
#                                                ylim = USA_ylim_meters,
#                                               )
# mode_map

In [ ]:
# trip the dataset to a smaller size so we can deploy the app
# modes = [3,1,7,16]
# trip_trim = trip_state_mode_geo[trip_state_mode_geo['mode'].isin(modes)]
# trip_trim.to_csv('trip_trim.csv', index=False)  

## Census Information Map
We want to pair the above trip mode map along with some demographic information including income, racial and demographic information. So that our user can compare them side by side. Here we will use the 2018 ACS 5 year estimates to match our trip data.

In [ ]:
import cenpy

In [ ]:
available = cenpy.explorer.available() # explore available datasets
cenpy.explorer.explain("ACSDT5Y2018")

In [ ]:
acs = cenpy.remote.APIConnection("ACSDT5Y2018")

In [ ]:
pd.set_option('display.max_colwidth', 80)
df = acs.varslike("RACE", by='concept').sort_index() 

In [ ]:
# After exploring the various tables, we decided to use the following ones

variables = [
    "NAME",
    "B21004_001E", #total median income
    "B17020_001E", #poverty total
    "B02001_001E", #total population
    "B02001_002E", #white alone
    "B02001_003E", #black or african americna alone
    "B02001_004E", #American Indian and Alaska Native alone
    "B02001_005E", #Asian alone
    "B15003_017E", #high school diploma
    "B15003_022E" #bachelor's degree
]

In [ ]:
state_acs = acs.query(
    cols=variables,
    geo_unit="state:*",
)

state_acs.head()

In [ ]:
# change column names so it can be more interpretable
state_acs.columns = ["NAME", "Median Income", "Poverty Population","Total Population",
    "White Population", "Black or African Americna Population", "American Indian and Alaska Native population",
    "Asian Population", "Population with High School Diploma", "Population with Bachelor's Degree", "state"]
state_acs.head()

Since hvplot allows a dropdown menu when we group values in one column, we need to melt the above dataframe from wide to long to allow users to select based on a drop down menu.

In [ ]:
# use melt function to turn the wide dataframe to a long dataframe
state_acs_long = pd.melt(state_acs, id_vars=['NAME','state'])
state_acs_long

From the above section, we got the geoboundaries from a geojson file downloaded online. Here, let's try out a new way and use Cenpy to look up for geographies.

In [ ]:
cenpy.tiger.available()

In [ ]:
acs.set_mapservice("tigerWMS_ACS2018")

In [ ]:
acs.mapservice.layers

In [ ]:
acs.mapservice.layers[84]

In [ ]:
# get boundaries by state
states_map = acs.mapservice.layers[84].query(where="1=1")

In [ ]:
states_map.head()

In [ ]:
states_map.plot()

Now we can merge the geographic informaiton with the demographic information.

In [ ]:
# merge state_acs and states_map based on state and STATE
state_acs_geo = state_acs_long.merge(states_map[['GEOID', 'STATE','geometry']], 
                                     left_on="state", right_on="STATE", how='left')
state_acs_geo =  gpd.GeoDataFrame(state_acs_geo, geometry='geometry', crs="EPSG:3857")
state_acs_geo["value"] = pd.to_numeric(state_acs_geo["value"])
state_acs_geo

Now our dataframe is ready to be plotted! We then use Hvplot to make a map that allows users to select which demographic variable to show on a map.

In [ ]:
# Get USA xlim and ylim in meters (EPSG=3857)
USA = ((-124.72,  -66.95), (23.55, 50.06))
USA_xlim_meters, USA_ylim_meters = [list(r) for r in lnglat_to_meters(USA[0], USA[1])]

acs_map = state_acs_geo.hvplot(c='value', 
                               frame_width=400, 
                               frame_height=300, 
                               groupby="variable",
                               geo=True,
                               crs=3857,
                               cmap='PuBu', 
                               hover_cols=['NAME'],
                               xlim=USA_xlim_meters, # Specify the xbounds in meters (EPSG=3857)
                               ylim=USA_ylim_meters
                              )
acs_map

In [ ]:
# mode_map+acs_map

## Build a webapp with Panel

We have our maps ready! Now we use Panel to create a dashboard for user to interact with. Furthermore, in the dashboard, we add in another altair bar chart to show the count of trips by the selected mode in each state.

In [ ]:
import panel as pn
pn.extension('vega') # This ensures altair works in the notebook!
import param as pm
from colorcet import cm

In [ ]:
# Add custom JS for altair here AFTER imports
# This ensures altair works when deploying on binder!
pn.extension(
    js_files={
        "vega": "https://cdn.jsdelivr.net/npm/vega@5/build/vega.min.js",
        "vegaLite": "https://cdn.jsdelivr.net/npm/vega-lite@4/build/vega-lite.min.js",
        "vegaEmbed": "https://cdn.jsdelivr.net/npm/vega-embed@6/build/vega-embed.min.js",
    }
)

In [ ]:
# Read in trimmed trip dataset
trip_state_mode_geo = gpd.read_file("./NHTS/Data/trip_trim.geojson")
trip_state_mode_geo.crs


In [ ]:
# The modes and demographic information users can choose from
mode_choice = trip_state_mode_geo['Trip Mode'].unique().tolist()
acs_choice = state_acs_geo["variable"].unique().tolist()

In [ ]:

# state_acs_geo.to_csv("state_acs.csv", index=False) 

The app has 4 components: 1) parameters and widgets, 2)map for demographic informaiton 3) map for trip mode information 4) bar chart for count of trips

In [ ]:
class TripMapApp(pm.Parameterized):
    """
    A Panel-based dashboard app visualizing data for NHTS trip mode 
    and ACS demographic informaiton.

    The app has three main components:
        1. A set of widgets controlling the data plotted on the two maps
        2. A map for demographic informatrion from ACS  
        3. A map for trip mode information from NHTS
        
    """
    #trip mode choice widget
    mode = pm.ObjectSelector(default="Walk", objects=mode_choice)
    #demographic widget
    demographic = pm.ObjectSelector(default="Median Income", objects=acs_choice)
    
    def filter_by_demo(self):
        demo_subset = state_acs_geo[state_acs_geo['variable']==self.demographic]
        return demo_subset
    
    def filter_by_mode(self):
        mode_subset = trip_state_mode_geo[trip_state_mode_geo['Trip Mode']==self.mode]
        return mode_subset

    #acs hvplot
    @pm.depends("demographic")
    def acsplot(self):
        """
        Return an hvplot for demographic information.
        """
        # get the filtered data
        acs_data = self.filter_by_demo()

        return acs_data.hvplot(c='value', 
                           frame_width=400, 
                           frame_height=300, 
                           geo=True,
                           crs=3857,
                           cmap='PuBu', 
                           hover_cols=['NAME'],
                           xlim=USA_xlim_meters, # Specify the xbounds in meters (EPSG=3857)
                           ylim=USA_ylim_meters
        ).opts(title="Demographic Information by State")
    
    #mode choice hvplot
    @pm.depends("mode")
    def modeplot(self):
        """
        Return an hvplot for demographic information.
        """
        # get the filtered data
        trip_data = self.filter_by_mode()

        return trip_data.hvplot.polygons(c='mode_share', 
                                    frame_width=400, 
                                    frame_height=300, 
                                    geo=True, 
                                    cmap='RdPu', 
                                    crs=3857,
                                    hover_cols=['state'],
                                    xlim = USA_xlim_meters, #add boundaries to make map centered on the U
                                    ylim = USA_ylim_meters,
                                   ).opts(title="Mode Share by State")
    @pm.depends("mode")
    def bar_chart(self):
        """
        Return an altair histogram of the number of trips using the selected mode choice.
        """
         # get the filtered data
        trip_data = self.filter_by_mode()

        # create the chart
        chart = (
            alt.Chart(trip_data[["state_mode_w", "state"]])
            .mark_bar()
            .encode(
                x=alt.X("state_mode_w",title='Count of trips'),
                y=alt.Y(
                    "state:N",title='State',
                    sort=alt.EncodingSortField(
                        field="state_mode_w",  # The field to use for the sort
                        order="descending",  # The order to sort in
                    ),
                ),
                tooltip=["state_mode_w", "state"],
            )
            .properties(width=300, height=500, title="Count of trips using the selected mode")
        )
        return chart

#add title for plots
#add total trips of the current mode is sum(state_mode_w)

In [ ]:
app = TripMapApp(name="")

In [ ]:
title = pn.Pane("<h1>Trip Mode and Demographics Distribution Maps</h1>", width=1000)

In [ ]:
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(pn.Param(app.param, width=300)),
    pn.Row(app.modeplot,app.bar_chart),
    pn.Row(app.acsplot),
)

In [ ]:
panel.servable()

The code below provides me two toy examples that can be put on static github pages.

In [ ]:
# toy static example for github pages

demo_toy = state_acs_geo[state_acs_geo['variable']=='Median Income']
mode_toy = trip_state_mode_geo[trip_state_mode_geo['Trip Mode']=='Walk']

trip_toy = mode_toy.hvplot.polygons(c='mode_share', 
                                    frame_width=400, 
                                    frame_height=300, 
                                    geo=True, 
                                    cmap='RdPu', 
                                    crs=3857,
                                    hover_cols=['state'],
                                    xlim = USA_xlim_meters, #add boundaries to make map centered on the U
                                    ylim = USA_ylim_meters,
                                   )

acs_toy = demo_toy.hvplot(c='value', 
                           frame_width=400, 
                           frame_height=300, 
                           geo=True,
                           crs=3857,
                           cmap='PuBu', 
                           hover_cols=['NAME'],
                           xlim=USA_xlim_meters, # Specify the xbounds in meters (EPSG=3857)
                           ylim=USA_ylim_meters
        )


In [ ]:
trip_toy.opts(title="Mode Share by State: Walk")

In [ ]:
acs_toy.opts(title="Demographic Information by State: Median Income")

In [ ]:
hvplot.save(acs_toy, 'acsHvplot.html')
hvplot.save(trip_toy, 'modeHvplot.html')